In [1]:
import sys
sys.path.append('../../')
from final_utils import *

In [2]:
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[1], True)
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.set_visible_devices(gpus[1], 'GPU')
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)


In [3]:
from ResNet_FC_joint import *

In [4]:
root_path = generate_root_path()

In [5]:
indxs = np.load('../../Genetic_Algorithm/best_res_all_res_996/best_solution_custom_95.72.npy')

def take_from_vector(data, indxs=indxs):
    new_data = np.squeeze(data[indxs])
    return new_data

### DataLoader and other nesseccary functions

In [6]:
# def lr_exp_decay(epoch, lr):
#     k = 0.04
#     return lr * np.exp(-k*epoch)

def make_dataset(paths, batch_size):
    main_path = f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
    def parse_image(filename):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32)
        print(image)
#         image = tf.image.resize(image, [996, 996]) #ToDo: check this, why we will resize img if we load alread resized img
        return image

    def configure_for_performance(ds):
        ds = ds.shuffle(buffer_size=1000)
        ds = ds.batch(batch_size)
        ds = ds.repeat()
        ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return ds
    for name in paths:
        img_basename = os.path.basename(name).split('.')[0] + '.npy'
        probs = np.load(root_path + f'Data/AesthAI/alm/splitted/alm_train/predictions/mg_cnn_fc/{img_basename}')
        if 'good' in name:
            label_img = 1
        else:
            label_img = 0
        feat = np.concatenate((np.squeeze(np.load(main_path + f'features/multigap/all_res_996/' + img_basename)), 
                              np.squeeze(np.load(main_path + f'features/cnn_efficientnet_b7/border_600x600/' + img_basename))))[indxs]
        labels = np.concatenate((feat, probs, np.array([label_img])))
    filenames_ds = tf.data.Dataset.from_tensor_slices(paths)
    images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((images_ds, labels_ds))
    ds = configure_for_performance(ds)

    return ds

### Trainer

In [7]:
def trainer(model, weights_path, train_paths, val_paths, batch_size=128, epochs=30, learning_rate=0.03, verbose=0):
    data = make_dataset(train_paths, batch_size)
    data_val = None#make_dataset(val_paths, batch_size)
    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, monitor='val_loss', verbose=1, save_best_only=True, 
                                                 mode='min')
    callbacks_list = [checkpoint]
    
   
    history = model.fit(data, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=data_val, 
                        steps_per_epoch=math.ceil(len(train_paths)/32), validation_steps=math.ceil(len(train_paths)/32), 
                        validation_batch_size=batch_size, callbacks=callbacks_list)

    return history

### Loading data

In [8]:
main_path = f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
paths_bad = []
paths_good = []
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    
    for data in bad_data:
        path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
        paths_bad.append(path_to_img)
        
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
    paths_good.append(path_to_img)
    
for i in range(7):
    paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
paths_good = np.squeeze(np.array(paths_good))
   
# Generating static validation data
paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.014) #original - 0.017
paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.06) #original - 0.11

paths_bad = np.array(paths_bad)
paths_bad_val = np.array(paths_bad_val)
paths_good = np.array(paths_good)
paths_good_val = np.array(paths_good_val)

In [9]:
full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
    
#shuffling
idx = np.random.permutation(len(full_data))
full_data = full_data[idx]
# full_data = full_data[:5000]  # debug
full_data.shape
paths_val = np.concatenate((paths_bad_val, paths_good_val ) , axis=0)

### Creating model and training

In [10]:
batch_size = 32 #64
epochs = 20
learning_rate = 0.009

In [11]:
# with tf.device('/GPU:0'):
model_res = ResNet18(num_classes=5000)
model_fc = fc_model_softmax(input_num=5000)
model = resnet_fc(model_res, model_fc)
model.build((None, 996, 996, 3))

# model.save_weights(weights_path)
model.summary()
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate))

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "resnet_fc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_net18_1 (ResNet18)      multiple                  4074504   
                                                                 
 model_2 (Functional)        (None, 2)                 12616450  
                                                                 
 model_1 (Functional)        (None, 2)                 5389058   
                                                                 
Total params: 22,080,012
Trainable params: 9,456,522
Non-trainable params: 12,623,490
_________________________________________________________________


In [12]:
inputs = keras.Input(shape=(996, 996, 3))
output = resnet_fc()
model = CustomModel(inputs, output(inputs))
model.compile(optimizer="adam")
weights_path = '../models/Resnet/resnet_fc_996_3l/'

In [13]:
history = trainer(model, weights_path, full_data[:512], paths_val, batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, 
                   verbose=1)    
    

Tensor("Cast:0", shape=(None, None, 3), dtype=float32)
Epoch 1/20
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("IteratorGetNext:0", shape=(None, None, None, 3), dtype=float32)


ValueError: in user code:

    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\CareAware\Consulting_Projects\aesthetic-ai\DeepFL\DeepFL\Distillation_Learning\tf_optimized\ResNet_FC_joint.py", line 64, in train_step
        feat_t = tf.slice(y, [0, 0], [32, 5000])

    ValueError: Shape must be rank 2 but is rank 1 for '{{node Slice}} = Slice[Index=DT_INT32, T=DT_DOUBLE](IteratorGetNext:1, Slice/begin, Slice/size)' with input shapes: [?], [2], [2].


In [ ]:
tf.slice(y, [-1])

In [ ]:
data = make_dataset(full_data[0], batch_size=5)

In [ ]:
list(data)